# SDTM Mapping Model based on Task Specific ELMo  
## Model 1

In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import sys
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten
from keras.models import Model
from keras.utils import to_categorical, np_utils
from keras.engine import Layer
#from keras.esimator import model_to_estimator
import numpy as np

from SDTMMapper import SDTMMapper, SDTMModels

bucket='snvn-sagemaker-1' #data bucket
KEY='mldata/Sam/data/project/xxx-000/xxx/xxx-201/csr/data/raw/latest/'
# Initialize session

tf.keras.backend.clear_session()
sess = tf.Session()
K.set_session(sess)


Using TensorFlow backend.


# Data

In [2]:
trdf = pd.read_csv('../train_data/ae_training.csv')
trdf.head()

,ID,text,sdtm
0,xxx203.AE.SUBJECTID,SUBJECTID INTERNAL ID FOR THE SUBJECT,DROP
1,xxx203.AE.AESTDTC_RAW,AESTDTC RAW START DATE (DD MMM YYYY) (RAW),DROP
2,xxx203.AE.AEACN,AEACN ACTION TAKEN WITH STUDY TREATMENT,AEACN
3,xxx203.AE.AETERM_LLT_CODE,AETERM LLT CODE AETERMLLT CODE,AELLTCD
4,xxx203.AE.AESER_STD,"AESER STD SERIOUS EVENT (IF 'YES', COMPLETE A ...",DROP


- Generate pickle for target SDTM vars used for decoder. 
- **Note:** This needs to be recreated when new class variables are added.


trdf['sdtm'].to_pickle('decode/ae.pkl')

Create 'clean' for training, and 'dropped' contains dropping variables that meet certain conditions Define suffix which is used to define what variables should be dropped compeltely from your SDTM. The specification will keep these variabels as 'drop'.

In [3]:
sdtmmap=SDTMMapper.SDTMMapper('ae', bucket, KEY)

Xt contains X for the models
df is the dataframe = original - dropped

In [4]:
suffix='.*_RAW$|.*_INT$|.*_STD$|.*_D{1,2}$|.*_M{1,2}$|.*_Y{1,4}|.*_CV$'
trdropped, X_train, trdf= sdtmmap.drop_sys_vars('../train_data/ae_training.csv', 'rave', suffix)
vdropped,  X_valid, vdf = sdtmmap.drop_sys_vars('../validation_data/ae_validation.csv', 'rave', suffix)
tdropped,  X_test,  tdf = sdtmmap.drop_sys_vars('../test_data/ae_test_study.csv', 'rave', suffix)

In [5]:
trdf.head()

,ID,text,sdtm
0,xxx203.AE.AEACN,AEACN ACTION TAKEN WITH STUDY TREATMENT,AEACN
1,xxx203.AE.AETERM_LLT_CODE,AETERM LLT CODE AETERMLLT CODE,AELLTCD
2,xxx203.AE.AEONGO,AEONGO ONGOING,AEENRTPT
3,xxx203.AE.AESTND,AESTND AESTND,DROP
4,xxx203.AE.AEENDTC,AEENDTC END DATE (DD MMM YYYY),AEENDTC


- Encode `Y` which contains sdtm variables fro training, validation, and testing
- Encoding can be decoded with `sdtmmap.decode_sdtm_target(Y)` later

In [6]:
y_train = sdtmmap.encode_sdtm_target(trdf['sdtm'].str.upper(), 'train_encode')
y_valid = sdtmmap.encode_sdtm_target(vdf['sdtm'].str.upper(), 'valid_encode')
y_test  = sdtmmap.encode_sdtm_target(tdf['sdtm'].str.upper(), 'test_encode')

In [7]:
#Check any difference in classes for training and validation set
print("extra in training:   ",[i for i in set(trdf['sdtm']) if i not in  set(vdf['sdtm'])])
print("extra in validation: ",[i for i in set(vdf['sdtm']) if i not in  set(trdf['sdtm'])])

extra in training:    []
extra in validation:  []


In [8]:
shape=len(set(trdf['sdtm']))
print("Number of classes in training: ",shape)
shape=len(set(vdf['sdtm']))
print("Number of classes in validation: ",shape)

Number of classes in training:  34
Number of classes in validation:  34


No need to do below
>- Clean ID variable (you may not need to do this)
>
> - Before cleaning:<br>
>   `xxx203.AE.AEACN`
> - After cleaning:<br>
>   `AEACN`
>
>- Create X which contains SAS variable names and SAS variable labels
>`df['ID']=df['ID'].str.replace('.*\..*\.','')`
>`X=df['ID'].str.lower()+" "+df['text'].str.lower()`

### Train and validate

X_train and y_train will be used for Training. 

X_valid and y_valid will be used for validation. 

18 studies for Training, 3 studies for Validation

In [9]:
X_train = np.array(X_train, dtype=object)[:, np.newaxis]
X_valid = np.array(X_valid, dtype=object)[:, np.newaxis]
X_test = np.array(X_test, dtype=object)[:, np.newaxis]

In [10]:
X_train.shape, X_valid.shape

((584, 1), (100, 1))

In [11]:
#df=np.array(df.iloc[:,1], dtype=object)[:, np.newaxis]

In [12]:
models=SDTMModels.SDTMModels('ae', shape)
model1 = models.build_model(1, False)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
filepath='checkpoint/chkpt_Elmo+sfnn+ae+Model1.hdf5'

checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

history = model1.fit(X_train, y=y_train, batch_size=32, verbose=1, validation_data=(X_valid,y_valid), 
          shuffle=True, epochs=5, callbacks=[checkpointer])

Train on 584 samples, validate on 100 samples
Epoch 1/5
584/584 [==============================] - 44s 75ms/step - loss: 2.8358 - acc: 0.3014 - val_loss: 1.7073 - val_acc: 0.7000

Epoch 00001: val_acc improved from -inf to 0.70000, saving model to checkpoint/chkpt_Elmo+sfnn+ae+Model1.hdf5
Epoch 2/5
584/584 [==============================] - 41s 70ms/step - loss: 1.5228 - acc: 0.6455 - val_loss: 0.8403 - val_acc: 0.8700

Epoch 00002: val_acc improved from 0.70000 to 0.87000, saving model to checkpoint/chkpt_Elmo+sfnn+ae+Model1.hdf5
Epoch 3/5
584/584 [==============================] - 43s 73ms/step - loss: 0.9001 - acc: 0.7945 - val_loss: 0.4695 - val_acc: 0.8800

Epoch 00003: val_acc improved from 0.87000 to 0.88000, saving model to checkpoint/chkpt_Elmo+sfnn+ae+Model1.hdf5
Epoch 4/5
584/584 [==============================] - 48s 82ms/step - loss: 0.5774 - acc: 0.8699 - val_loss: 0.3030 - val_acc: 0.9700

Epoch 00004: val_acc improved from 0.88000 to 0.97000, saving model to checkpoint/

In [14]:
model1.save('../model/Elmo+sfnn+ae+Model1.h5')

# Test Results

In [16]:
def macro_acc_test_data(model, X_=X_test, testdf=tdf, droppeddf=tdropped ):
    
    predictions = model1.predict(X_)
    testdf['pred']=sdtmmap.decode_sdtm_target(predictions, 'train_encode')
    df=sdtmmap.add_drop(testdf,droppeddf)
    return df, sum(df['sdtm']==df['pred'])/len(df)

df, acc=macro_acc_test_data(model1)
print("macro acc: ",acc)

macro acc:  0.9080459770114943


### Check where the model made mistake

In [17]:
df[df['sdtm']!=df['pred']]

,ID,text,sdtm,pred
4,LLT_NAME,LLT_NAME LLT_NAME,AELLT,DROP
5,LLT_CODE,LLT_CODE LLT_CODE,AELLTCD,DROP
7,PT_CODE,PT_CODE PT_CODE,AEPTCD,DROP
8,HLT_NAME,HLT_NAME HLT_NAME,AEHLT,DROP
9,HLT_CODE,HLT_CODE HLT_CODE,AEHLTCD,AEHLGT
13,SOC_CODE,SOC_CODE SOC_CODE,AESOCCD,DROP
17,AEENTIM,AEENTIM Stop Time,AEENDTC_TM,DROP
32,AESTTIMU,AESTTIMU Start Time Unknown,DROP,AESTDTC_TM


In [18]:
df.head()

,ID,text,sdtm,pred
0,SUBJECT,SUBJECT Subject name or identifier,SUBJID,SUBJID
1,INSTANCENAME,INSTANCENAME Folder instance name,DROP,DROP
2,AETERM,AETERM Reported Term for the Adverse Event,AETERM,AETERM
3,VMEDDRA,VMEDDRA MedDRA Version Num,DROP,DROP
4,LLT_NAME,LLT_NAME LLT_NAME,AELLT,DROP


In [19]:
tf.keras.backend.clear_session()
